In [1]:
# how to kill all processes
# sudo kill -9 $(pgrep -f python3)
# sudo kill -9 $(pgrep -f conformanceTests)

# import subprocess
# subprocess.run(["kill" "-9" $(pgrep -f conformanceTests)"])
import os
os.system("kill -9 $(pgrep -f conformanceTests)")

9

In [14]:
!pip install xlsxwriter==3.0.2
!pip install beautifulsoup4==4.10.0
!pip install utils==1.0.1

In [11]:
import os
python_exec = '/home/nsemaev/code/venv/bin/python3.8'

work_path = os.getcwd()

binary_type = 'master_2022_02_12'
binary_name = f"conformanceTests_{binary_type}"
binary_path = f"{work_path}/bin/{binary_name}"


ci_report_file = f"{work_path}/ci_reports/"+"2022_02_08.html"
report_plugin_position = 4

results_path = f"{work_path}/{binary_type}"

ping_time = 60 * 5

irs_path = '/home/nsemaev/Documents/_ops/'

skipped_ops = ['boolean', 'ROIAlign']
time_wasting_ops = ['BinaryConvolution', 'Convolution', 'ConvolutionBackpropData', 'DeformableConvolution', 
                    'GroupConvolution', 'GroupConvolutionBackpropData', 'MaxPool', 'AvgPool', 'Interpolate',
                    'TensorIterator']

all_ops = sorted(os.listdir(irs_path))

run_results = ['passed', 'failed']
ci_results = ['passed', 'failed', 'skipped', 'crashed']

In [13]:
import os
from os import path
import sys
import time
from datetime import datetime
import shutil
import subprocess
import _thread
import threading
from contextlib import contextmanager
import re
import xlsxwriter
from bs4 import BeautifulSoup


completed_ops = []
if os.path.exists(results_path):
    completed_ops = [op.split('_')[0] for op in os.listdir(results_path) if op.split('_')[-1] == 'completed']
logs_files = {}

class GTestParallel():
    def __init__(self, op: str):
        self.op = op
        self.op_path = f"{results_path}/{self.op}_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}"
        self.op_completed_path = f"{self.op_path}_completed"
        self.op_stopped_path = f"{self.op_path}_stopped"
        self.command = f"{python_exec} {work_path}/gtest_parallel.py {binary_path} -d . " + \
            f"--gtest_filter=*ReadIRTest*{self.op}* " + \
            f"-- " + \
            f"--input_folders={irs_path}/{self.op} " + \
            f"--device=TEMPLATE " + \
            f"--report_unique_name " + \
            f"--output_folder={self.op_path}/gtest-parallel-xmls"
        self.report_command = f"{python_exec} {work_path}/merge_xmls.py -i ./gtest-parallel-xmls/ -o ./"

    def run_while_not_end(self, op_time=30, time_limited=True):
        process_start_time = time.time()
        current_time = datetime.now().strftime('%Y_%m_%d %H:%M:%S')
        print(f"{current_time} {self.op} was started")
        print(self.command)
        if os.path.exists(self.op_path):
            shutil.rmtree(self.op_path)
        os.makedirs(self.op_path)
        os.chdir(self.op_path)
        threading.Thread(target=os.system, args=(f"cd {self.op_path} && {self.command}",)).start()
        # _thread.start_new_thread(os.system, (f"cd {self.op_path} && nohup {self.command}",))
        os.chdir(work_path)
        # statuses = ['passed', 'failed']
        start_time = time.time()
        last_ping = time.time()
        while True:
            time.sleep(1)
            if (os.path.exists(f"{self.op_path}/gtest-parallel-logs/passed")):
                break
            if (os.path.exists(f"{self.op_path}/gtest-parallel-logs/failed")):
                break
            if time_limited and (time.time() - start_time > op_time):
                print(f"{datetime.now().strftime('%Y_%m_%d %H:%M:%S')} Sorry, time for operation {self.op} is over")
                os.rename(self.op_path, self.op_stopped_path)
                os.system('kill -9 $(pgrep -f conformanceTests)')
                return False
            if time.time() - last_ping > ping_time:
                print(f"{datetime.now().strftime('%Y_%m_%d %H:%M:%S')} {self.op} ping")
                last_ping = time.time()
        
        log_files_folder = f"{self.op_path}/gtest-parallel-logs/failed"
        failed_logs_result = f"{self.op_path}/{self.op}_failed_logs_result.txt"
        if os.path.exists(log_files_folder):
            with open(failed_logs_result, 'w', encoding='utf-8') as result_file:
                for log_file_name in os.listdir(log_files_folder):
                    log_file_path = os.path.join(log_files_folder, log_file_name)  # !!! везде так же сделать

                    # print(f"\n\n{log_file_path}\n\n")

                    with open(log_file_path, 'r', encoding='utf-8') as file:
                        data = str(file.read())
                        test_filter = '/'.join(re.findall(r"Google Test filter = ([^\n]+)", data))
                        mem_usage = '/'.join(re.findall(r"MEM_USAGE=([^\n]+)", data))
                        failures = '/'.join(re.findall(r"[^\n]+pp:\s*\d+[^\n]+", data))
                        # print(f"\n\n{test_filter}")
                        # print(f"\n\n{mem_usage}")
                        # print(f"\n\n{failures}")
                        result_file.write(test_filter + ',')
                        result_file.write(mem_usage + ',')
                        result_file.write(failures + ';\n')
                    # with open(log_path, 'r', encoding='utf-8') as file:
                    #     print(file.read())

        print(f"cd {self.op_path} && {self.report_command}")
        threading.Thread(target=os.system, args=(f"cd {self.op_path} && {self.report_command}",)).start()
        while (not os.path.exists(f"{self.op_path}/report.xml")):
            pass
        print(f"{datetime.now().strftime('%Y_%m_%d %H:%M:%S')} {self.op} was completed")
        self.op_completed_path = f"{self.op_path}_{int(time.time() - process_start_time)}s_completed"
        os.rename(self.op_path, self.op_completed_path)
        return True
# TODO: get report_file by last name in work path
def generate_run_data():
    data = {}
    for i, op in enumerate(all_ops):
        run_folders = [folder for folder in os.listdir(results_path) 
                    if folder.startswith(op) and folder.endswith('completed')]
        print(run_folders)
        if run_folders:
            run_folder = sorted(run_folders)[-1]
            run_path = f"{results_path}/{run_folder}"
            data[op] = {}
            for result in run_results:
                result_path = f"{run_path}/gtest-parallel-logs/{result}"
                data[op][result] = len(os.listdir(result_path)) if path.exists(result_path) else 0

            failed_report = f"{run_path}/{op}_failed_logs_result.txt"
            if path.exists(failed_report):
                logs_files[op] = failed_report
        else:
            data[op] = {result: 'untested' for result in run_results}
    print('\n\n\n\nokokok\n\n\n')
    print(data)
    return data

def generate_ci_data():
    data = {}
    with open(ci_report_file, 'r') as file:
        soup = BeautifulSoup(str(file.read()), 'html.parser')
    tbody = soup.findAll('tbody')[1]
    for op_tr in tbody.findAll('tr'):
        op = op_tr.findAll('th')[0].text.split('-')[0]
        template_td = op_tr.findAll('td')[report_plugin_position]
        data[op] = {result: 'untested' for result in ci_results}
        for result in data[op].keys():
            if len(template_td.find_all('span')) == 4:
                data[op][result] = int(re.findall(fr"{result[0].upper()}:(\d+)", template_td.text)[0])
    return data

def get_from_TensorIterator_report(op: str):
    result = 'TensorIterator not tested'
    folders = [folder for folder in os.listdir(results_path) 
        if folder.split('_')[0] == 'TensorIterator' and folder.split('_')[-1] == 'completed']
    if folders:
        report_path = f"{results_path}/{sorted(folders)[-1]}/report.xml"
        with open(report_path, 'r', encoding='utf-8') as file:
            re_str = rf'<{op}-\d+ passed="\d+" failed="\d+" skipped="\d+" crashed="\d+" passrate="\d+\.\d+" />'
            result = ''.join(re.findall(re_str, file.read()))    
    return result

def generate_xlsx():
    columns = ['Operation', 'TensorIterator', 'Info', 'Group'] + \
                [f"RUN {result}" for result in run_results] + \
                [f"CI {result}" for result in ci_results] + ['RUN logs']
    columns_len = [len(column) for column in columns]
    run_data = generate_run_data()
    ci_data = generate_ci_data()
    ops = sorted(list(set(list(run_data.keys()) + list(ci_data.keys()))))

    # TODO: Delete operations that have 100% passrate in both lists, a better algorithm
    bad_ops = []
    for op in ops:
        flag = False
        if op in run_data:
            for result in run_results:
                if result not in ['passed']:
                    if run_data[op][result] != 'untested' and int(run_data[op][result]) > 0:
                        flag = True
                        break
        if op in ci_data:
            for result in ci_results:
                if result not in ['passed']:
                    if ci_data[op][result] != 'untested' and int(ci_data[op][result]) > 0:
                        flag = True
                        break
        if flag:
            bad_ops.append(op)
    ops = bad_ops

    workbook_path = f"{results_path}/table_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}.xlsx"
    workbook = xlsxwriter.Workbook(workbook_path)
    worksheet = workbook.add_worksheet('report')
    for i, column in enumerate(columns):
        worksheet.write(0, i, column, workbook.add_format({"bold": True}))
    for i, op in enumerate(ops):
        shift = 0
        worksheet.write(i + 1, shift, op)
        columns_len[shift] = max(columns_len[shift], len(str(op)))
        shift += 1
        data = get_from_TensorIterator_report(op)
        if data is not None:
            worksheet.write(i + 1, shift, data)
        shift += 3
        if op in run_data:
            for j, result in enumerate(run_results):
                worksheet.write(i + 1, j + shift, run_data[op][result])
        shift += len(run_results)
        if op in ci_data:
            for j, result in enumerate(ci_results):
                worksheet.write(i + 1, j + shift, ci_data[op][result])
        shift += len(ci_results)
        if op in logs_files:
            with open(logs_files[op], 'r', encoding='utf-8') as file:
                worksheet.write(i + 1, shift, ''.join(file.readlines()))


    for i, column_len in enumerate(columns_len):
        if column_len > 0:
            worksheet.set_column(i, i, column_len)
    workbook.close()

def generate_full_report():
    if not os.path.exists(results_path):
        return
    folders = [folder for folder in os.listdir(results_path) if folder.split('_')[-1] == 'completed']
    folders = sorted(folders)
    ops = [folder.split('_')[0] for folder in folders]
    unique_folders, unique_ops = [], []
    for i, folder in enumerate(folders):
        if ops[i] not in ops[i+1:]:
            unique_folders.append(folder)
            unique_ops.append(ops[i])
    reports_path = f"{results_path}/reports"
    if os.path.exists(reports_path):
        shutil.rmtree(reports_path)
    os.makedirs(reports_path)
    for i, folder in enumerate(unique_folders):
        shutil.copy(f"{results_path}/{folder}/report.xml", f"{reports_path}/report_{unique_ops[i]}.xml")
    full_report_name = f"full_report_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}_{len(unique_ops)}pcs.xml"
    gen_report_command = f"{python_exec} {work_path}/merge_xmls.py -i {reports_path} -o {results_path}"
    os.system(gen_report_command)
    os.rename(f"{results_path}/report.xml", f"{results_path}/{full_report_name}")
    shutil.rmtree(reports_path)
    return f"{results_path}/{full_report_name}"
    
    

if __name__ == '__main__':
    ops = [op for op in all_ops if op not in skipped_ops + completed_ops + time_wasting_ops]
    ops = ['Add']
    print(f"Ops for run: {ops}")
    for op in ops:
        GTestParallel(op).run_while_not_end(time_limited=False)
#     generate_full_report()
#     generate_xlsx()
    
#         generate_xlsx()
#     generate_xlsx()
        

    # print(generate_run_data())
    # print(generate_ci_data())
    # generate_xlsx()


Ops for run: ['Add']
2022_02_12 01:52:24 Add was started
/home/nsemaev/code/venv/bin/python3.8 /home/nsemaev/code/ConformanceTests/gtest_parallel.py /home/nsemaev/code/ConformanceTests/bin/conformanceTests_master_2022_02_12 -d . --gtest_filter=*ReadIRTest*Add* -- --input_folders=/home/nsemaev/Documents/_ops//Add --device=TEMPLATE --report_unique_name --output_folder=/home/nsemaev/code/ConformanceTests/master_2022_02_12/Add_2022_02_12_01_52_24/gtest-parallel-xmls
[0/4] Running tests...
[1/4] conformance/ReadIRTest.ReadIR/PRC=Add_1269_IR_name=Add_1269.xml_TargetDevice=TEMPLATE_Config=() (78 ms)
[2/4] conformance/ReadIRTest.ReadIR/PRC=Add_2064_IR_name=Add_206486.xml_TargetDevice=TEMPLATE_Config=() (87 ms)
[3/4] conformance/ReadIRTest.ReadIR/PRC=Add_5005_IR_name=Add_50056.xml_TargetDevice=TEMPLATE_Config=() (112 ms)
[4/4] conformance/ReadIRTest.ReadIR/PRC=Add_1345_IR_name=Add_1345.xml_TargetDevice=TEMPLATE_Config=() (114 ms)
cd /home/nsemaev/code/ConformanceTests/master_2022_02_12/Add_2022

INFO:XmlMerger: Processing is finished
INFO:XmlMerger: Processing: ./gtest-parallel-xmls/report_51424_1644619944987640907.xml
INFO:XmlMerger: Processing: ./gtest-parallel-xmls/report_51419_1644619944988685587.xml
INFO:XmlMerger: Processing: ./gtest-parallel-xmls/report_51421_1644619944961201121.xml
INFO:XmlMerger: Processing: ./gtest-parallel-xmls/report_51423_1644619944954344895.xml
INFO:XmlMerger: Processing is finished
INFO:XmlMerger: Final report is saved to file: './report.xml'
INFO:XmlMerger: Processing is finished
ERROR:XmlMerger:/home/nsemaev/code/ConformanceTests/master_2022_02_12/reports/report_Add.xml is incorrect! Error to get a xml root
ERROR:XmlMerger:/home/nsemaev/code/ConformanceTests/master_2022_02_12/reports does not contain the correct xml files
Traceback (most recent call last):
  File "/home/nsemaev/code/ConformanceTests/merge_xmls.py", line 114, in <module>
    merge_xml(arguments.input_folders, arguments.output_folder, arguments.output_filename)
  File "/home/nse

FileNotFoundError: [Errno 2] No such file or directory: '/home/nsemaev/code/ConformanceTests/master_2022_02_12/report.xml' -> '/home/nsemaev/code/ConformanceTests/master_2022_02_12/full_report_2022_02_12_01_52_25_1pcs.xml'

In [1]:
print("hi")

hi


In [7]:
os.system("$(pgrep -f conformanceTests)")

sh: 1: 69938: not found


32512

In [34]:
number = 4
data = {}
with open(report_file, 'r') as file:
    soup = BeautifulSoup(str(file.read()), 'html.parser')
tbody = soup.findAll('tbody')[1]
for op_tr in tbody.findAll('tr'):
    op = op_tr.findAll('th')[0].text.split('-')[0]
    template_td = op_tr.findAll('td')[number]
    print(op)
    data[op] = {result: 'untested' for result in ci_results}
    for result in data[op].keys():
        print(result)
        if len(template_td.find_all('span')) == 4:
            data[op][result] = int(re.findall(fr"{result[0].upper()}:(\d+)", template_td.text)[0])
print(data)
# return data

Abs
passed
failed
skipped
crashed
Acos
passed
failed
skipped
crashed
Acosh
passed
failed
skipped
crashed
AdaptiveAvgPool
passed
failed
skipped
crashed
AdaptiveMaxPool
passed
failed
skipped
crashed
Add
passed
failed
skipped
crashed
Asin
passed
failed
skipped
crashed
Asinh
passed
failed
skipped
crashed
Assign
passed
failed
skipped
crashed
Assign
passed
failed
skipped
crashed
Atan
passed
failed
skipped
crashed
Atanh
passed
failed
skipped
crashed
AvgPool
passed
failed
skipped
crashed
BatchNormInference
passed
failed
skipped
crashed
BatchNormInference
passed
failed
skipped
crashed
BatchToSpace
passed
failed
skipped
crashed
BinaryConvolution
passed
failed
skipped
crashed
Broadcast
passed
failed
skipped
crashed
Broadcast
passed
failed
skipped
crashed
Bucketize
passed
failed
skipped
crashed
CTCGreedyDecoder
passed
failed
skipped
crashed
CTCGreedyDecoderSeqLen
passed
failed
skipped
crashed
CTCLoss
passed
failed
skipped
crashed
Ceiling
passed
failed
skipped
crashed
Clamp
passed
failed
skipped
cr